In [1]:
!pip install kaggle

In [5]:
!kaggle datasets download -d mbornoe/lisa-traffic-light-dataset

Dataset URL: https://www.kaggle.com/datasets/mbornoe/lisa-traffic-light-dataset
License(s): CC-BY-NC-SA-4.0
100%|███████████████████████████████████████| 4.21G/4.21G [00:19<00:00, 298MB/s]
100%|███████████████████████████████████████| 4.21G/4.21G [00:19<00:00, 228MB/s]


In [6]:
import zipfile

with zipfile.ZipFile("lisa-traffic-light-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("lisa_traffic_light")

In [7]:
import os

print(os.listdir("lisa_traffic_light"))

['nightTrain', 'sample-dayClip6', 'sample-nightClip1', 'dayTrain', 'Annotations', 'nightSequence1', 'daySequence1', 'nightSequence2', 'daySequence2']


In [17]:
!pip install -U ultralytics torch torchvision torchaudio pandas scikit-learn pyyaml tqdm opencv-python-headless Pillow wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 973.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 11.1 MB/s eta 0:00:00:00:0100:01
   ━━━

In [4]:
# --- Installation (only run once) ---
!pip install ultralytics pandas scikit-learn pyyaml tqdm opencv-python-headless Pillow # Headless OpenCV for Kaggle
!pip install wandb # For live visualization

# --- Imports ---
import os
import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
import shutil
import cv2
from tqdm.notebook import tqdm # Use notebook-friendly tqdm
import torch
import wandb
import sys
import time
from ultralytics import YOLO
import logging

# --- Configure Logging ---
# Reduce ultralytics logging spam if needed, or keep default for verbosity
# logging.getLogger('ultralytics').setLevel(logging.WARNING)

print("--- Setup Complete ---")
# Verify GPU availability for Ultralytics/PyTorch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("WARNING: No CUDA GPUs detected. Training will use CPU.")
print("-" * 30)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.3/977.3 kB 14.6 MB/s eta 0:00:00a 0:00:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
--- Setup Complete ---
PyTorch version: 2.5.1+cu121
CUDA available: True
Number of GPUs: 1
  GPU 0: Tesla P100-PCIE-16GB
------------------------------


In [6]:
# --- Dataset Paths (Adjust if your unzip location is different) ---
BASE_DIR = "/kaggle/working/"
LISA_DATASET_DIR = os.path.join(BASE_DIR, "lisa_traffic_light")
ANNOTATIONS_DIR = os.path.join(LISA_DATASET_DIR, "Annotations")
# !! IMPORTANT: Verify this CSV filename exists in the Annotations folder !!
# Common names: allAnnotations.csv, annotations.csv. Check your folder!
LISA_CSV_PATH = os.path.join(ANNOTATIONS_DIR, "allAnnotations.csv")
# Base directory where actual image frames are stored (the script will look inside subfolders like daySequence1/frames)
LISA_IMAGES_BASE_DIR = LISA_DATASET_DIR # Images are relative to this path based on CSV Filename column

# --- Output Paths ---
YOLO_DATASET_NAME = "lisa_yolo_format"
YOLO_DATASET_DIR = os.path.join(BASE_DIR, YOLO_DATASET_NAME)
OUTPUT_LABELS_DIR = os.path.join(YOLO_DATASET_DIR, "raw_labels") # Temporary location for all converted labels
TRAIN_IMG_DIR = os.path.join(YOLO_DATASET_DIR, "images/train")
VAL_IMG_DIR = os.path.join(YOLO_DATASET_DIR, "images/val")
TRAIN_LABEL_DIR = os.path.join(YOLO_DATASET_DIR, "labels/train")
VAL_LABEL_DIR = os.path.join(YOLO_DATASET_DIR, "labels/val")
DATA_YAML_PATH = os.path.join(YOLO_DATASET_DIR, "data.yaml")

# --- Data Preparation Parameters ---
# !! CRITICAL: Define your class mapping !!
# Map LISA annotation tags (lowercase) to YOLO class IDs (0, 1, 2...).
# VERIFY these tags against the unique values in the 'Annotation tag' column of your CSV!
CLASS_MAPPING = {
    'stop': 0,
    'stopleft': 0,  # Map to 'stop'
    'warning': 1,
    'warningleft': 1, # Map to 'warning'
    'go': 2,
    'goleft': 2,    # Map to 'go'
    'goforward': 2, # Map to 'go'
    # Add 'off' or others if they appear and you need them
}

print(f"Using Class Mapping: {CLASS_MAPPING}")

VALIDATION_SPLIT_SIZE = 0.20 # 20% of data for validation
RANDOM_STATE_SPLIT = 42 # For reproducible train/val splits

# --- YOLOv8 Training Parameters ---
MODEL_CHOICE = 'yolov8n.pt' # Nano version - good for edge devices (Raspberry Pi/Jetson)
EPOCHS = 75 # Start with 50-100, adjust based on results
IMG_SIZE = 640
BATCH_SIZE = 32 # Adjust based on T4 memory (16/32 common for 2xT4) - If CUDA OOM, reduce this!
PATIENCE_EPOCHS = 15 # Early stopping patience
OPTIMIZER = 'AdamW' # Or 'SGD', 'Adam'
LEARNING_RATE = 0.001 # Example LR, can tune this (YOLO defaults are often good)
NUM_WORKERS = 2 # Kaggle typically has 2 CPU cores available

PROJECT_NAME = 'LISA_Traffic_Light_Detection'
RUN_NAME = f'{MODEL_CHOICE.split(".")[0]}_e{EPOCHS}_bs{BATCH_SIZE}_{time.strftime("%Y%m%d_%H%M%S")}'

# --- Device Config ---
# Automatically use available GPUs, fallback to CPU
# if torch.cuda.is_available():
#     gpu_count = torch.cuda.device_count()
#     if gpu_count >= 2:
#         DEVICE_CONFIG = '0,1' # Use first two GPUs
#         print("Configuring training for 2 GPUs (0, 1).")
#     elif gpu_count == 1:
#         DEVICE_CONFIG = '0'   # Use the single available GPU
#         print("Configuring training for 1 GPU (0).")
#     else: # This case should ideally not happen if cuda.is_available is true, but safety first
#          DEVICE_CONFIG = 'cpu'
#          print("CUDA reported available, but no devices found? Falling back to CPU.")
# else:
#     DEVICE_CONFIG = 'cpu'
#     print("Configuring training for CPU.")

if torch.cuda.is_available():
    DEVICE_CONFIG = '0' # Use only GPU 0
    print("***** DEBUGGING: Forcing training on SINGLE GPU (0) *****")
else:
    DEVICE_CONFIG = 'cpu'
    print("Configuring training for CPU.")
# ***** END MODIFICATION *****

print(f"Training Device Setting: {DEVICE_CONFIG}")
print(f"Output dataset dir: {YOLO_DATASET_DIR}")
print("-" * 30)

print(f"Training Device Setting: {DEVICE_CONFIG}")
print(f"Output dataset dir: {YOLO_DATASET_DIR}")
print("-" * 30)
print("--- Cell 2 finished defining variables ---")
print(f"DEBUG: MODEL_CHOICE in Cell 2 is: {MODEL_CHOICE}") # Verify one variable

Using Class Mapping: {'stop': 0, 'stopleft': 0, 'warning': 1, 'warningleft': 1, 'go': 2, 'goleft': 2, 'goforward': 2}
***** DEBUGGING: Forcing training on SINGLE GPU (0) *****
Training Device Setting: 0
Output dataset dir: /kaggle/working/lisa_yolo_format
------------------------------
Training Device Setting: 0
Output dataset dir: /kaggle/working/lisa_yolo_format
------------------------------
--- Cell 2 finished defining variables ---
DEBUG: MODEL_CHOICE in Cell 2 is: yolov8n.pt


In [7]:
print(MODEL_CHOICE)
print(DATA_YAML_PATH)

yolov8n.pt
/kaggle/working/lisa_yolo_format/data.yaml


In [19]:
import glob # Import glob for finding files
import os
import pandas as pd
import cv2
from tqdm.notebook import tqdm # Use notebook-friendly tqdm
import sys

print("--- Starting Annotation Conversion (CSV to YOLO .txt) ---")

# --- Create Output Directories ---
# (Directories should exist from previous run, exist_ok=True handles it)
os.makedirs(OUTPUT_LABELS_DIR, exist_ok=True)
os.makedirs(TRAIN_IMG_DIR, exist_ok=True)
os.makedirs(VAL_IMG_DIR, exist_ok=True)
os.makedirs(TRAIN_LABEL_DIR, exist_ok=True)
os.makedirs(VAL_LABEL_DIR, exist_ok=True)
print(f"Ensured output directories exist inside: {YOLO_DATASET_DIR}")

# --- Find all relevant Annotation CSV files ---
search_pattern = os.path.join(ANNOTATIONS_DIR, 'Annotations', '*', 'frameAnnotationsBOX.csv')
print(f"Searching for annotation files using pattern: {search_pattern}")
csv_files = glob.glob(search_pattern)

if not csv_files:
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(f"ERROR: No 'frameAnnotationsBOX.csv' files found using pattern: {search_pattern}")
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    sys.exit("No annotation CSV files found. Stopping.")
else:
    print(f"Found {len(csv_files)} annotation CSV files to process.")

# --- Load and Combine Annotations ---
all_annotations_list = []
print("Loading and combining annotations...")
for csv_path in tqdm(csv_files, desc="Reading CSVs"):
    try:
        sequence_name = os.path.basename(os.path.dirname(csv_path))
        temp_df = pd.read_csv(csv_path, delimiter=';')
        temp_df['sequence'] = sequence_name
        temp_df['csv_source'] = csv_path
        all_annotations_list.append(temp_df)
    except Exception as e:
        print(f"\nWARNING: Failed to load or parse CSV file '{csv_path}': {e}. Skipping.")

if not all_annotations_list:
     print("ERROR: Failed to load data from any CSV file. Stopping.")
     sys.exit("No annotation data loaded.")

df = pd.concat(all_annotations_list, ignore_index=True)
print(f"Combined {len(df)} total annotation entries from {len(csv_files)} files.")

# --- Display Combined CSV Info ---
# print("\nFirst 5 rows of the combined annotation data:") # Optional: uncomment to view
# print(df.head())
print("\nUnique values in 'Annotation tag' column:")
if 'Annotation tag' in df.columns:
    print(df['Annotation tag'].dropna().unique())
else:
    print("'Annotation tag' column not found!")
print("\nColumns found in combined data:", df.columns.tolist())
print("-" * 30)

# --- Verify Required Columns (Adjust names if needed based on printout above!) ---
required_cols = ['Filename', 'Annotation tag', 'Upper left corner X', 'Upper left corner Y', 'Lower right corner X', 'Lower right corner Y', 'sequence']
if not all(col in df.columns for col in required_cols):
    print(f"ERROR: Missing one or more required columns in the combined CSV data!")
    print(f"Script Requires: {required_cols}. Found: {df.columns.tolist()}")
    sys.exit("Missing required columns in combined CSV data. Stopping.")
print("Required columns found in combined data.")

# --- Conversion Process ---
conversion_errors = 0
files_processed_map = {} # Track files {img_basename: [yolo_line, ...]}
annotations_converted_count = 0
skipped_classes = set()
processed_image_paths = {} # Store mapping {img_basename: {'path': str, 'h': int, 'w': int}}

print(f"Processing {len(df)} annotation entries...")

# Iterate through rows of the combined dataframe
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Converting Annotations"):

    sequence_name = row['sequence'] # e.g., 'daySequence1'
    filename_in_csv = row['Filename'] # e.g., 'dayTest/daySequence1--00111.jpg'

    # --- *** CORRECTED Image Path Construction *** ---
    # Extract the actual filename part (e.g., daySequence1--00111.jpg)
    actual_image_filename = os.path.basename(filename_in_csv)
    # Construct the path based on observed structure: base/seq/seq/frames/actual_file.jpg
    image_path = os.path.normpath(os.path.join(LISA_IMAGES_BASE_DIR, sequence_name, sequence_name, 'frames', actual_image_filename))
    # --- ***************************************** ---

    # Get image basename (e.g., 'daySequence1--00111') - Use this as the key
    image_basename = os.path.splitext(actual_image_filename)[0]

    # Get image dimensions (read only once per image)
    if image_basename not in processed_image_paths:
        if not os.path.isfile(image_path):
            # This is now the expected failure point if path logic is still wrong
            # print(f"\nDEBUG: Image file not found at constructed path: '{image_path}' (derived from seq='{sequence_name}', csv_filename='{filename_in_csv}')")
            conversion_errors += 1
            # Store None to avoid re-checking this failed path
            processed_image_paths[image_basename] = {'path': None, 'h': None, 'w': None}
            continue # Skip this annotation if image doesn't exist at constructed path

        # Try reading the image only if path exists
        try:
            img = cv2.imread(image_path)
            if img is None:
                print(f"\nWarning: Found but failed to read image '{image_path}'. Skipping annotations for it.")
                conversion_errors += 1
                processed_image_paths[image_basename] = {'path': image_path, 'h': None, 'w': None}
                continue
            img_height, img_width, _ = img.shape
            if img_height == 0 or img_width == 0:
                 print(f"\nWarning: Invalid image dimensions (0) for '{image_path}'. Skipping.")
                 conversion_errors += 1
                 processed_image_paths[image_basename] = {'path': image_path, 'h': None, 'w': None}
                 continue
            # Store path and dimensions upon successful read
            processed_image_paths[image_basename] = {'path': image_path, 'h': img_height, 'w': img_width}

        except Exception as e:
            print(f"\nERROR reading image '{image_path}': {e}")
            conversion_errors += 1
            processed_image_paths[image_basename] = {'path': image_path, 'h': None, 'w': None}
            continue
    # End if image_basename not in processed_image_paths

    # Retrieve stored data
    img_data = processed_image_paths[image_basename]
    img_height = img_data['h']
    img_width = img_data['w']

    # Skip if we failed to get dimensions or path earlier
    if img_data['path'] is None or img_height is None or img_width is None:
        continue

    # --- Process the annotation for this row (Class Mapping) ---
    class_label = str(row['Annotation tag']).lower().strip()

    # Use the CLASS_MAPPING defined in Cell 2
    if class_label not in CLASS_MAPPING:
        if class_label not in skipped_classes:
            # print(f"\nInfo: Skipping unrecognized class label '{class_label}' (in {row['csv_source']} for {filename_in_csv}). Update CLASS_MAPPING in Cell 2 if needed.")
            skipped_classes.add(class_label)
        continue # Skip if class not in our defined mapping

    class_id = CLASS_MAPPING[class_label]

    # Get bounding box coordinates
    try:
        x1 = int(row['Upper left corner X'])
        y1 = int(row['Upper left corner Y'])
        x2 = int(row['Lower right corner X'])
        y2 = int(row['Lower right corner Y'])
    except ValueError as ve:
        # print(f"\nWarning: Invalid coordinate value for '{class_label}' in {row['csv_source']} for {filename_in_csv}: {ve}. Skipping box.")
        conversion_errors += 1
        continue

    # Coordinate Validation (same as before)
    valid_coords = True
    if x1 >= x2 or y1 >= y2 or x1 < 0 or y1 < 0 or x2 > img_width or y2 > img_height:
        valid_coords = False; conversion_errors += 1
    if not valid_coords:
        continue

    # Convert to YOLO format (same as before)
    box_width = float(x2 - x1); box_height = float(y2 - y1)
    center_x = float(x1) + box_width / 2; center_y = float(y1) + box_height / 2
    norm_center_x = round(center_x / img_width, 6); norm_center_y = round(center_y / img_height, 6)
    norm_width = round(box_width / img_width, 6); norm_height = round(box_height / img_height, 6)
    norm_center_x = max(0.0, min(1.0, norm_center_x)); norm_center_y = max(0.0, min(1.0, norm_center_y))
    norm_width    = max(0.0, min(1.0, norm_width)); norm_height   = max(0.0, min(1.0, norm_height))
    if norm_width <= 0 or norm_height <= 0:
        conversion_errors += 1; continue

    # Add annotation line to the map for this image basename
    if image_basename not in files_processed_map:
        files_processed_map[image_basename] = []
    files_processed_map[image_basename].append(f"{class_id} {norm_center_x} {norm_center_y} {norm_width} {norm_height}")
    annotations_converted_count += 1


# --- Write YOLO .txt files ---
print(f"\nWriting YOLO label files to {OUTPUT_LABELS_DIR}...")
labels_written_count = 0
# Filter map to only include entries where annotations were successfully generated
valid_entries_for_writing = {k: v for k, v in files_processed_map.items() if v}
print(f"Attempting to write {len(valid_entries_for_writing)} label files.")

for img_basename, yolo_lines in tqdm(valid_entries_for_writing.items(), desc="Writing Label Files"):
    label_filename = f"{img_basename}.txt"
    label_path = os.path.join(OUTPUT_LABELS_DIR, label_filename)
    try:
        with open(label_path, 'w') as f:
            f.write("\n".join(yolo_lines) + "\n")
        labels_written_count += 1
    except Exception as e:
        print(f"\nERROR: Failed to write label file '{label_path}': {e}")
        conversion_errors += 1


# --- Final Conversion Summary ---
print("\n--- Annotation Conversion Summary ---")
# Count how many images were successfully read (had dimensions)
successful_reads = sum(1 for data in processed_image_paths.values() if data['h'] is not None)
print(f"Images processed (found file and read dimensions): {successful_reads}")
print(f"Total unique image basenames encountered: {len(processed_image_paths)}")
print(f"Annotations converted successfully (associated with a read image): {annotations_converted_count}")
print(f"YOLO label files written: {labels_written_count} (to {OUTPUT_LABELS_DIR})")
if skipped_classes:
    print(f"Skipped unrecognized classes (Check CLASS_MAPPING in Cell 2): {list(skipped_classes)}")
# Recalculate errors more accurately: Total annotations - successful conversions
total_possible_conversions = len(df) # Number of rows we tried to process
final_error_count = total_possible_conversions - annotations_converted_count
print(f"Encountered approx {final_error_count} warnings/errors (incl. missing images, bad coords/reads, skipped classes, write errors).")
print("--- Annotation Conversion Finished ---")
print("-" * 30)

# --- Update processed_image_paths for Train/Val Split ---
# We need a simple map from basename -> full image path for the next cell
# Ensure we only include paths for images that were successfully processed
image_path_mapping_for_split = {basename: data['path']
                                for basename, data in processed_image_paths.items()
                                if data['path'] is not None and data['h'] is not None}
print(f"Created image path map with {len(image_path_mapping_for_split)} entries for Train/Val split.")
if len(image_path_mapping_for_split) == 0 and labels_written_count > 0:
    print("\nCRITICAL WARNING: Label files were written, but the image path map for splitting is empty. Check logic.")
elif len(image_path_mapping_for_split) == 0:
     print("\nWARNING: No images were successfully processed. Train/Val split will likely fail.")

--- Starting Annotation Conversion (CSV to YOLO .txt) ---
Ensured output directories exist inside: /kaggle/working/lisa_yolo_format
Searching for annotation files using pattern: /kaggle/working/lisa_traffic_light/Annotations/Annotations/*/frameAnnotationsBOX.csv
Found 4 annotation CSV files to process.
Loading and combining annotations...


Reading CSVs:   0%|          | 0/4 [00:00<?, ?it/s]

Combined 57649 total annotation entries from 4 files.

Unique values in 'Annotation tag' column:
['go' 'stop' 'warning' 'stopLeft' 'goLeft' 'warningLeft' 'goForward']

Columns found in combined data: ['Filename', 'Annotation tag', 'Upper left corner X', 'Upper left corner Y', 'Lower right corner X', 'Lower right corner Y', 'Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number', 'sequence', 'csv_source']
------------------------------
Required columns found in combined data.
Processing 57649 annotation entries...


Converting Annotations:   0%|          | 0/57649 [00:00<?, ?it/s]


Writing YOLO label files to /kaggle/working/lisa_yolo_format/raw_labels...
Attempting to write 18252 label files.


Writing Label Files:   0%|          | 0/18252 [00:00<?, ?it/s]


--- Annotation Conversion Summary ---
Images processed (found file and read dimensions): 18252
Total unique image basenames encountered: 18252
Annotations converted successfully (associated with a read image): 57649
YOLO label files written: 18252 (to /kaggle/working/lisa_yolo_format/raw_labels)
Encountered approx 0 warnings/errors (incl. missing images, bad coords/reads, skipped classes, write errors).
--- Annotation Conversion Finished ---
------------------------------
Created image path map with 18252 entries for Train/Val split.


In [21]:
import os
import shutil
import sys
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

print("--- Starting Train/Validation Split ---")

# Get list of image basenames for which we generated labels
# Make sure 'image_path_mapping_for_split' exists from the end of Cell 3
if 'image_path_mapping_for_split' not in locals() or not isinstance(image_path_mapping_for_split, dict):
     print("ERROR: 'image_path_mapping_for_split' dictionary not found or is not a dictionary.")
     print("Please ensure Cell 3 ran correctly and created this variable.")
     sys.exit("Aborting split due to missing input map.")

valid_basenames = list(image_path_mapping_for_split.keys()) # Use the corrected map from Cell 3
print(f"Found {len(valid_basenames)} images with corresponding annotations to split.")

if not valid_basenames:
    print("ERROR: No valid image basenames found in the map from Cell 3. Cannot proceed.")
    sys.exit("Aborting due to lack of processable data.")

# Split the basenames
try:
    train_basenames, val_basenames = train_test_split(
        valid_basenames,
        test_size=VALIDATION_SPLIT_SIZE, # Defined in Cell 2
        random_state=RANDOM_STATE_SPLIT  # Defined in Cell 2
    )
except Exception as e:
     print(f"ERROR during train_test_split: {e}")
     sys.exit("Aborting split.")

print(f"Splitting into: {len(train_basenames)} Train / {len(val_basenames)} Validation images")

# --- Function to copy files ---
def copy_files(basenames, source_img_path_map, source_label_dir, dest_img_dir, dest_label_dir):
    copied_count = 0
    error_count = 0
    print(f"Copying files to {dest_img_dir} and {dest_label_dir}...")
    for basename in tqdm(basenames, desc=f"Copying to {os.path.basename(dest_img_dir)}"):
        # Get the actual image source path from the map created in Cell 3
        img_src_path = source_img_path_map.get(basename) # This should now be a string path
        label_src_path = os.path.join(source_label_dir, f"{basename}.txt")

        # --- *** CORRECTED LOGIC *** ---
        if img_src_path and os.path.exists(label_src_path):
            # Now img_src_path should be a string, directly usable
            if not isinstance(img_src_path, str):
                 print(f"\nWarning: Expected string path for basename '{basename}', but got {type(img_src_path)}. Skipping.")
                 error_count += 1
                 continue

            img_filename = os.path.basename(img_src_path) # Get filename from the string path
            label_filename = f"{basename}.txt"          # Label filename uses basename

            img_dest_path = os.path.join(dest_img_dir, img_filename)
            label_dest_path = os.path.join(dest_label_dir, label_filename)

            try:
                # Ensure source image file exists before copying
                if not os.path.isfile(img_src_path):
                     print(f"\nWarning: Source image file confirmed missing at '{img_src_path}' for basename '{basename}'. Skipping copy.")
                     error_count +=1
                     continue

                shutil.copy2(img_src_path, img_dest_path) # copy2 preserves metadata
                shutil.copy2(label_src_path, label_dest_path)
                copied_count += 1
            except Exception as e:
                print(f"\nError copying files for basename '{basename}' (Image: '{img_src_path}', Label: '{label_src_path}'): {e}")
                error_count += 1
        # --- ************************* ---
        else:
             # Log reasons for skipping
             if not img_src_path:
                 print(f"\nWarning: Image source path not found in map for basename '{basename}'. Cannot copy.")
             elif not os.path.exists(label_src_path):
                  print(f"\nWarning: Label file not found at '{label_src_path}' for basename '{basename}'. Cannot copy image either.")
             # Increment error count if either is missing
             error_count += 1
             
    print(f"Finished copying. Copied: {copied_count}, Errors/Skipped: {error_count}")
    return copied_count

# --- Copy Training Files ---
# Use the 'image_path_mapping_for_split' dictionary created at the end of Cell 3
train_copied = copy_files(train_basenames, image_path_mapping_for_split, OUTPUT_LABELS_DIR, TRAIN_IMG_DIR, TRAIN_LABEL_DIR)

# --- Copy Validation Files ---
val_copied = copy_files(val_basenames, image_path_mapping_for_split, OUTPUT_LABELS_DIR, VAL_IMG_DIR, VAL_LABEL_DIR)

# --- Split Summary ---
print("\n--- Train/Validation Split Summary ---")
print(f"Total images intended for train: {len(train_basenames)}, Copied successfully: {train_copied}")
print(f"Total images intended for val:   {len(val_basenames)}, Copied successfully: {val_copied}")
# Check if the number of successfully copied files matches the intended number
train_errors = len(train_basenames) - train_copied
val_errors = len(val_basenames) - val_copied
if train_errors > 0 or val_errors > 0:
     print(f"Warning: {train_errors} training files and {val_errors} validation files were NOT copied successfully. Check warnings above.")
print(f"Training images copied to:   {TRAIN_IMG_DIR}")
print(f"Training labels copied to:   {TRAIN_LABEL_DIR}")
print(f"Validation images copied to: {VAL_IMG_DIR}")
print(f"Validation labels copied to: {VAL_LABEL_DIR}")
print("--- Train/Validation Split Finished ---")
print("-" * 30)

--- Starting Train/Validation Split ---
Found 18252 images with corresponding annotations to split.
Splitting into: 14601 Train / 3651 Validation images
Copying files to /kaggle/working/lisa_yolo_format/images/train and /kaggle/working/lisa_yolo_format/labels/train...


Copying to train:   0%|          | 0/14601 [00:00<?, ?it/s]

Finished copying. Copied: 14601, Errors/Skipped: 0
Copying files to /kaggle/working/lisa_yolo_format/images/val and /kaggle/working/lisa_yolo_format/labels/val...


Copying to val:   0%|          | 0/3651 [00:00<?, ?it/s]

Finished copying. Copied: 3651, Errors/Skipped: 0

--- Train/Validation Split Summary ---
Total images intended for train: 14601, Copied successfully: 14601
Total images intended for val:   3651, Copied successfully: 3651
Training images copied to:   /kaggle/working/lisa_yolo_format/images/train
Training labels copied to:   /kaggle/working/lisa_yolo_format/labels/train
Validation images copied to: /kaggle/working/lisa_yolo_format/images/val
Validation labels copied to: /kaggle/working/lisa_yolo_format/labels/val
--- Train/Validation Split Finished ---
------------------------------


In [22]:
print("--- Creating data.yaml ---")

# --- Get Class Names in Order ---
# Sort the CLASS_MAPPING by ID (0, 1, 2...) to ensure correct order in YAML
sorted_class_items = sorted(CLASS_MAPPING.items(), key=lambda item: item[1])
class_names = [item[0] for item in sorted_class_items]
num_classes = len(class_names)

print(f"Number of classes: {num_classes}")
print(f"Class names (in order): {class_names}")

# --- Create YAML Data Structure ---
yaml_data = {
    'train': TRAIN_IMG_DIR,
    'val': VAL_IMG_DIR,
    'nc': num_classes,
    'names': class_names
}

# --- Write YAML File ---
try:
    with open(DATA_YAML_PATH, 'w') as f:
        yaml.dump(yaml_data, f, sort_keys=False) # sort_keys=False preserves order
    print(f"Successfully created data.yaml at: {DATA_YAML_PATH}")
    # --- Print YAML content for verification ---
    print("\n--- data.yaml content: ---")
    with open(DATA_YAML_PATH, 'r') as f:
        print(f.read())
    print("--------------------------")
except Exception as e:
    print(f"ERROR: Failed to write data.yaml file: {e}")
    sys.exit("Aborting due to YAML write error.")

print("--- data.yaml Creation Finished ---")
print("-" * 30)

--- Creating data.yaml ---
Number of classes: 7
Class names (in order): ['stop', 'stopleft', 'warning', 'warningleft', 'go', 'goleft', 'goforward']
Successfully created data.yaml at: /kaggle/working/lisa_yolo_format/data.yaml

--- data.yaml content: ---
train: /kaggle/working/lisa_yolo_format/images/train
val: /kaggle/working/lisa_yolo_format/images/val
nc: 7
names:
- stop
- stopleft
- warning
- warningleft
- go
- goleft
- goforward

--------------------------
--- data.yaml Creation Finished ---
------------------------------


In [ ]:
# --- Imports for Cell 7 ---
import time
import sys
import os
import glob
from IPython.display import Image, display
from ultralytics import YOLO
import torch
import traceback
# --- End Imports ---

print("--- Cell 7 Starting ---")
# --- Variable Existence Check (Alternative Method) ---
print("DEBUG: Checking required variables individually...")
variables_ok = True
required_vars_list = ['MODEL_CHOICE', 'DATA_YAML_PATH', 'EPOCHS', 'BATCH_SIZE', 'IMG_SIZE',
                      'DEVICE_CONFIG', 'PROJECT_NAME', 'RUN_NAME', 'BASE_DIR',
                      'PATIENCE_EPOCHS', 'NUM_WORKERS', 'OPTIMIZER', 'LEARNING_RATE']
missing_vars_details = []

for var_name in required_vars_list:
    # Check if the variable exists in either the local or global scope
    if var_name not in locals() and var_name not in globals():
        print(f"DEBUG: Variable '{var_name}' NOT FOUND.")
        missing_vars_details.append(var_name)
        variables_ok = False
    # else: # Optional: uncomment to confirm variables are found
    #     print(f"DEBUG: Variable '{var_name}' found.")

if not variables_ok:
    print(f"ERROR: The following configuration variables are not defined: {missing_vars_details}")
    print("Please ensure Cell 2 (Configuration) was run successfully before this cell.")
    sys.exit("Aborting training due to missing config variables.")
else:
     print("DEBUG: All required variables confirmed present.")
# --- End Variable Check ---

print("\n--- Starting YOLOv8 Training ---") # Add newline for clarity

# --- Print Configuration (Now that variables are confirmed) ---
print(f"Model: {MODEL_CHOICE}")
print(f"Dataset Config: {DATA_YAML_PATH}")
print(f"Epochs: {EPOCHS}, Batch Size: {BATCH_SIZE}, Image Size: {IMG_SIZE}")
print(f"Device: {DEVICE_CONFIG}")
print(f"Project: {PROJECT_NAME}, Run Name: {RUN_NAME}")
print(f"Patience: {PATIENCE_EPOCHS}, Workers: {NUM_WORKERS}, Optimizer: {OPTIMIZER}, LR: {LEARNING_RATE}")
# Construct full results path for clarity
results_dir = os.path.join(BASE_DIR, PROJECT_NAME, RUN_NAME)
print(f"Output will be saved to: {results_dir}")

# --- W&B Status (Variable defined in Cell 6) ---
if 'wandb_enabled' not in locals():
     wandb_enabled = False # Assume disabled if Cell 6 wasn't run or failed
     print("W&B status unknown (assuming DISABLED).")
elif wandb_enabled:
    print("Weights & Biases logging is ENABLED.")
else:
    print("Weights & Biases logging is DISABLED.")
print("-" * 30)

os.environ['NCCL_DEBUG'] = 'INFO' # Get more verbose output from NCCL (helps debugging if it hangs)
os.environ['NCCL_SOCKET_IFNAME'] = 'eth0' # Sometimes needed in Docker - Kaggle usually uses 'eth0'
print("DEBUG: Set NCCL_DEBUG=INFO and NCCL_SOCKET_IFNAME=eth0")

# --- Load Model ---
try:
    model = YOLO(MODEL_CHOICE) # Load pretrained weights
    print(f"Loaded model '{MODEL_CHOICE}' successfully.")
except Exception as e:
    print(f"ERROR: Failed to load YOLO model: {e}")
    sys.exit("Aborting training.")


# --- Start Training ---
start_time = time.time()
training_successful = False
print("\n--- Initiating model.train() ---")
print("Training progress (loss, metrics per epoch) will be printed below by Ultralytics:")
print("-" * 60)
try:
    # The train function handles multi-GPU and W&B integration automatically
    # It will print epoch, loss, mAP, etc. automatically to the output
    results = model.train(
        data=DATA_YAML_PATH,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        device=DEVICE_CONFIG,
        project=os.path.join(BASE_DIR, PROJECT_NAME), # Save results inside /kaggle/working/
        name=RUN_NAME,
        patience=PATIENCE_EPOCHS,
        workers=NUM_WORKERS,
        optimizer=OPTIMIZER,
        lr0=LEARNING_RATE,
        # exist_ok=False, # Prevent overwriting existing runs by default
        # val=True, # Validate after each epoch (default)
        # save=True, # Save checkpoints (default)
    )
    print("-" * 60)
    print("\n--- model.train() Completed ---")
    training_successful = True

except Exception as e:
    print("-" * 60)
    print(f"\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(f"ERROR: An exception occurred during YOLOv8 training:")
    traceback.print_exc() # Print detailed traceback
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    training_successful = False # Ensure it's marked as failed

except KeyboardInterrupt: # Handle manual interruption (e.g., pressing stop button)
     print("-" * 60)
     print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
     print("KeyboardInterrupt: Training stopped manually.")
     print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
     training_successful = False # Mark as incomplete


# --- Training Summary & Visualization ---
end_time = time.time()
duration = end_time - start_time
print(f"\n--- Training Attempt Finished ---")
print(f"Total time elapsed for this attempt: {duration:.2f} seconds")

if training_successful:
    print(f"\n✅ Training completed successfully.")
    print("\nTraining results (model weights, logs, plots) saved in:")
    # results_dir was defined earlier
    best_model_path = os.path.join(results_dir, 'weights', 'best.pt')
    print(f" -> Directory: '{results_dir}'")
    print(f" -> Best model weights (for inference/deployment): '{best_model_path}'")
    if os.path.exists(best_model_path):
         print("   (Best model file confirmed to exist)")
    else:
         print("   (Warning: Best model file 'best.pt' not found in expected location - check logs)")

    # --- *** INTEGRATED VISUALIZATION *** ---
    print("\n--- Displaying Training Results Visualizations ---")
    print(f"Looking for plots in: {results_dir}")

    plot_files = {
        "Results (Loss & Metrics)": os.path.join(results_dir, "results.png"),
        "Confusion Matrix": os.path.join(results_dir, "confusion_matrix.png"),
        "Precision-Recall Curve": os.path.join(results_dir, "PR_curve.png"),
        "Precision Curve": os.path.join(results_dir, "P_curve.png"),
        "Recall Curve": os.path.join(results_dir, "R_curve.png"),
        "F1 Curve": os.path.join(results_dir, "F1_curve.png"),
    }

    found_plots = False
    for plot_name, plot_path in plot_files.items():
        if os.path.exists(plot_path):
            print(f"\n-> {plot_name}:")
            try:
                display(Image(filename=plot_path, width=800)) # Adjust width as needed
                found_plots = True
            except Exception as img_e:
                 print(f"   Error displaying image '{plot_path}': {img_e}")
        else:
            print(f"   Plot not found: {os.path.basename(plot_path)}") # Print only filename if not found

    if not found_plots:
         print("\nCould not find standard plot files (results.png, etc.) in the results directory.")

    # Display sample validation predictions if available
    val_batch_labels = sorted(glob.glob(os.path.join(results_dir, "val_batch*_labels.jpg")))
    val_batch_preds = sorted(glob.glob(os.path.join(results_dir, "val_batch*_pred.jpg")))

    if val_batch_labels and val_batch_preds:
         print("\n-> Sample Validation Batch Predictions (Batch 0):")
         try:
             print("   Ground Truth Labels:")
             display(Image(filename=val_batch_labels[0], width=800))
             print("   Model Predictions:")
             display(Image(filename=val_batch_preds[0], width=800))
         except Exception as img_e:
              print(f"   Error displaying validation batch images: {img_e}")
    else:
        print("\nValidation batch images (val_batch*_labels.jpg / val_batch*_pred.jpg) not found.")
    # --- ******************************** ---

else:
    print(f"\n❌ Training did not complete successfully (or was interrupted).")
    print("Results directory might be incomplete or missing.")

# --- W&B Finish (Conditional) ---
# Check wandb_enabled again in case Cell 6 defined it differently
if 'wandb_enabled' in locals() and wandb_enabled and training_successful:
    print("\nAttempting to finish W&B run...")
    try:
         # Check if wandb was actually imported and initialized before finishing
         if 'wandb' in sys.modules and wandb.run is not None:
              wandb.finish()
              print("W&B run finished.")
         else:
              print("W&B run was not active, skipping finish.")
    except Exception as wb_e:
         print(f"Error finishing W&B run: {wb_e}")

print("\n--- Cell 7 Execution Finished ---")
print("-" * 30)

--- Cell 7 Starting ---
DEBUG: Checking required variables individually...
DEBUG: All required variables confirmed present.

--- Starting YOLOv8 Training ---
Model: yolov8n.pt
Dataset Config: /kaggle/working/lisa_yolo_format/data.yaml
Epochs: 75, Batch Size: 32, Image Size: 640
Device: 0
Project: LISA_Traffic_Light_Detection, Run Name: yolov8n_e75_bs32_20250404_100024
Patience: 15, Workers: 2, Optimizer: AdamW, LR: 0.001
Output will be saved to: /kaggle/working/LISA_Traffic_Light_Detection/yolov8n_e75_bs32_20250404_100024
W&B status unknown (assuming DISABLED).
------------------------------
DEBUG: Set NCCL_DEBUG=INFO and NCCL_SOCKET_IFNAME=eth0
Loaded model 'yolov8n.pt' successfully.

--- Initiating model.train() ---
Training progress (loss, metrics per epoch) will be printed below by Ultralytics:
------------------------------------------------------------
Ultralytics 8.3.101 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.3MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      75/75      4.24G     0.8366     0.3865     0.8198         27        640: 100%|██████████| 457/457 [02:40<00:00,  2.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 58/58 [00:20<00:00,  2.89it/s]

                   all       3651      11576      0.929      0.875      0.936       0.64



75 epochs completed in 3.973 hours.
Optimizer stripped from /kaggle/working/LISA_Traffic_Light_Detection/yolov8n_e75_bs32_20250404_100024/weights/last.pt, 6.2MB
Optimizer stripped from /kaggle/working/LISA_Traffic_Light_Detection/yolov8n_e75_bs32_20250404_100024/weights/best.pt, 6.2MB

Validating /kaggle/working/LISA_Traffic_Light_Detection/yolov8n_e75_bs32_20250404_100024/weights/best.pt...
Ultralytics 8.3.101 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  47%|████▋     | 27/58 [00:11<00:13,  2.32it/s]